In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker
from google.cloud import bigquery
import os
from google.cloud import bigquery


# pd.set_option('display.max_columns', None)

os.chdir('/home/jupyter')
!pwd
# # %cd 
# !pwd
# (base) jupyter@starfire-1:~$ pip install jupyter-console
# (base) jupyter@starfire-1:~$ !jupyter console --existing
%connect_info



/home/jupyter
{
  "shell_port": 56147,
  "iopub_port": 42571,
  "stdin_port": 38193,
  "control_port": 45597,
  "hb_port": 58121,
  "ip": "127.0.0.1",
  "key": "0e28c7e1-b44733e48760ea8e02d3cc22",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": "conda-root-py",
  "jupyter_session": "/home/jupyter/starfire/notebooks/refinitiv_api.ipynb"
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-4dc5f9ad-85cb-4862-9775-6f7f9abfb6db.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [8]:
# RDP_LOGIN = '<RDP Machine ID>'
# RDP_PASSWORD = '<RDP Password>'
# APP_KEY = '<APP Key>'

import refinitiv.dataplatform as rdp

%run ./starfire/notebooks/credentials.ipynb

session = rdp.open_platform_session(
    APP_KEY,
    rdp.GrantPassword(
        username = RDP_LOGIN,
        password = RDP_PASSWORD
    )
)

# https://developers.lseg.com/en/article-catalog/article/getting-start-with-refinitiv-data-platform

In [16]:
rdp.get_data(
    universe=["TRI.N", "IBM.N"],
    fields=["TR.Revenue", "TR.GrossProfit"]
)

,instrument,TR.Revenue,TR.GrossProfit
0,TRI.N,6794000000,6490000000
1,IBM.N,61860000000,34300000000


In [17]:
rdp.get_data(["AAPL.O","MSFT.O"],fields=["TR.F.MktCap"])


,instrument,TR.F.MktCap
0,AAPL.O,2676736860720
1,MSFT.O,3321869074628.3


In [10]:
rdp.convert_symbols(
    ['US5949181045', 'US02079K1079', 'US4592001014'], 
    from_symbol_type='ISIN') #TICKER
rdp.convert_symbols(
    ['.SPX'], 
    from_symbol_type='RIC')

,DocumentTitle,IssuerOAPermID,RIC,CUSIP
.SPX,"S&P 500 Index - CBOE, Equity Index",8589934169,.SPX,78378X107


In [13]:
# import datetime
# import pytz

# # Initialize the ses
# stock_symbol = 'AAPL.O'  # Example stock symbol for Apple Inc.
# start_date = datetime.datetime(2023, 6, 26, tzinfo=pytz.UTC)  # Example start date with timezone
# end_date = datetime.datetime(2023, 6, 26, tzinfo=pytz.UTC)  # Example end date with timezone
# interval = rdp.Intervals.DAILY
# fields = ['BID', 'ASK', 'OPEN_PRC', 'HIGH_1', 'LOW_1', 'TRDPRC_1', 'NUM_MOVES', 'TRNOVR_UNS']

# response = rdp.get_historical_price_summaries(
#     universe = [stock_symbol],
#     interval = interval,
#     start = start_date,
#     end = end_date,
#     fields = fields
# )


rdp.get_historical_price_summaries(
    universe = '.SPX', 
    interval = rdp.Intervals.DAILY,
    start = "2020-01-15", 
    end = "2020-01-29",
)


In [64]:
print(datetime.timedelta(0))

<module 'datetime' from '/opt/conda/lib/python3.10/datetime.py'>


In [20]:
mna_screen = f'SCREEN(U(IN(DEALS)/*UNV:DEALSMNA*/),\
                    BETWEEN(TR.MnAAnnDate,20000101,20241231)/*dt:Date*/, \
                    IN(TR.MnAAcquiror, "TRIMBLE NAVIGATION LTD"),\
                    NOT_IN(TR.MnAAcquirorNation, "CN"),\
                    NOT_IN(TR.MnAStatus, "S", "R", "DR", "W", "SW"), \
                    CURN=USD)'


        # input ipo screen results as instrument list and pull ipo attributes via fields from Eikon Data API. 
mnas = rdp.get_data(mna_screen, ['TR.MnASDCDealNumber',
                               'TR.MnAAnnDate',
                               'TR.MnARankValueIncNetDebt(Curn=USD,Scale=6)',
                               'TR.MnADealValue(Curn=USD,Scale=6)', 'TR.MnADealCurrency',
                               'TR.MnADealType',
                               'TR.MnAType',
                               'TR.MnATarget', 'TR.MnATargetNation', 'TR.MnATargetSDCCusip',
                               'TR.MnAAcquiror', 'TR.MnAAcquirorNation', 'TR.MnAAcquirorSDCCusip',
                               'TR.MnAAcquirorUltParent', 'TR.MnAAcquirorUltParentSDCCusip',
                               'TR.MnAPctHeldAtAnnDate', 'TR.MnAPctOfSharesAcquired', 'TR.MnAPctOfSharesAcqSeeksToOwn',
                               'TR.MnAStatus',
                               'TR.MnACompDealId(Concat="|")']) 

print(mnas.shape)
mnas

(97, 21)


,instrument,TR.MNASDCDealNumber,TR.MnAAnnDate,TR.MnARankValueIncNetDebt,TR.MnADealValue,TR.MnADealCurrency,TR.MnADealType,TR.MnAType,TR.MnATarget,TR.MnATargetNation,...,TR.MnAAcquiror,TR.MnAAcquirorNation,TR.MnAAcquirorSDCCusip,TR.MnAAcquirorUltParent,TR.MnAAcquirorUltParentSDCCusip,TR.MnAPctHeldAtAnnDate,TR.MnAPctOfSharesAcquired,TR.MnAPctOfSharesAcqSeeksToOwn,TR.MnAStatus,TR.MnACompDealId
0,34391219589,1005549020,2000-05-11,294.0,294.0,US Dollar,Disclosed Dollar Value Deal,Disclosed Value,Spectra Precision Inc,United States,...,Trimble Navigation Ltd,United States,896239,Trimble Navigation Ltd,896239,<NA>,100.0,100,Completed,
1,34391224495,1051097020,2000-09-21,15.0,15.0,US Dollar,Disclosed Dollar Value Deal,Disclosed Value,Tripod Data System Inc{TDS},United States,...,Trimble Navigation Ltd,United States,896239,Trimble Navigation Ltd,896239,<NA>,0.0,100,Pending,
2,34391244403,1357768020,2002-06-28,6.0,6.0,US Dollar,Disclosed Dollar Value Deal,Disclosed Value,LeveLite Technology Inc,United States,...,Trimble Navigation Ltd,United States,896239,Trimble Navigation Ltd,896239,<NA>,100.0,100,Completed,
3,34391254559,1498950020,2004-02-17,<NA>,<NA>,US Dollar,Undisclosed Dollar Value Deal,Undisclosed Value,TracerNet Corp,United States,...,Trimble Navigation Ltd,United States,896239,Trimble Navigation Ltd,896239,<NA>,100.0,100,Completed,
4,34391262279,1626471020,2004-12-21,<NA>,<NA>,US Dollar,Undisclosed Dollar Value Deal,Undisclosed Value,Pacific Crest Corp,United States,...,Trimble Navigation Ltd,United States,896239,Trimble Navigation Ltd,896239,<NA>,0.0,100,Pending,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,154082967973,2818735040,2015-11-10,<NA>,<NA>,Canadian Dollar,Undisclosed Dollar Value Deal,Undisclosed Value,Agri-Trend Data Corp,Canada,...,Trimble Navigation Ltd,United States,896239,Trimble Navigation Ltd,896239,<NA>,0.0,100,Pending,
93,154082968769,2819290020,2015-11-11,<NA>,<NA>,US Dollar,Undisclosed Dollar Value Deal,Undisclosed Value,Telog Instruments Inc,United States,...,Trimble Navigation Ltd,United States,896239,Trimble Navigation Ltd,896239,<NA>,100.0,100,Completed,
94,154083005207,2918733040,2016-02-08,<NA>,<NA>,British Pound Sterling,Undisclosed Dollar Value Deal,Undisclosed Value,Sefaira Ltd,United Kingdom,...,Trimble Navigation Ltd,United States,896239,Trimble Navigation Ltd,896239,<NA>,100.0,100,Completed,
95,154083096302,2989728040,2016-08-01,<NA>,<NA>,Euro,Undisclosed Dollar Value Deal,Undisclosed Value,AXIO-NET GmbH,Germany,...,Trimble Navigation Ltd,United States,896239,Trimble Navigation Ltd,896239,<NA>,100.0,100,Completed,


ERROR!!! Failed to request a new token......
         Token requested error b'{"error":"invalid_grant"   } '
ERROR!!! request a new refresh token has been failed.
The server-mode is disabled, so we do not re-authorize with username/password.
ERROR!!! authentication handler raise an exception.
ValueError('ERROR!!! request a new refresh token has been failed.\nThe server-mode is disabled, so we do not re-authorize with username/password.')


In [16]:
print(mnas['TR.MnAType'].value_counts())
print(mnas['TR.MnADealType'].value_counts())
mnas[mnas['TR.MnATarget'].str.contains('Yamei')]


TR.MnAType
Undisclosed Value                     78
Disclosed Value                       12
Repurchases                            3
Tender Offers                          2
Acquisitions of Remaining Interest     1
Privatizations                         1
Name: count, dtype: Int64
TR.MnADealType
Undisclosed Dollar Value Deal    78
Disclosed Dollar Value Deal      12
Repurchases Deal                  3
Name: count, dtype: Int64


,instrument,TR.MNASDCDealNumber,TR.MnAAnnDate,TR.MnARankValueIncNetDebt,TR.MnADealValue,TR.MnADealCurrency,TR.MnADealType,TR.MnAType,TR.MnATarget,TR.MnATargetNation,...,TR.MnAAcquiror,TR.MnAAcquirorNation,TR.MnAAcquirorSDCCusip,TR.MnAAcquirorUltParent,TR.MnAAcquirorUltParentSDCCusip,TR.MnAPctHeldAtAnnDate,TR.MnAPctOfSharesAcquired,TR.MnAPctOfSharesAcqSeeksToOwn,TR.MnAStatus,TR.MnACompDealId
54,34392053993,2319296040,2011-06-14,<NA>,<NA>,Chinese Yuan,Undisclosed Dollar Value Deal,Undisclosed Value,Changchun Yamei Electronics Technology Co Ltd,China (Mainland),...,Trimble Navigation Ltd,United States,896239,Trimble Navigation Ltd,896239,<NA>,100.0,100,Completed,


ERROR!!! Failed to request a new token......
         Token requested error b'{"error":"invalid_grant"   } '
ERROR!!! request a new refresh token has been failed.
The server-mode is disabled, so we do not re-authorize with username/password.
ERROR!!! authentication handler raise an exception.
ValueError('ERROR!!! request a new refresh token has been failed.\nThe server-mode is disabled, so we do not re-authorize with username/password.')


In [10]:

# Initialize a BigQuery client
client = bigquery.Client()

# Define your project, dataset, and table
project_id = "starfire-410116"
dataset_id = "derived"
table_id = "refinitiv_deals_2"


# Construct a reference to the target table
table_ref = client.dataset(dataset_id).table(table_id)

# Configure the load job to append data to the existing table
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND
)


from datetime import datetime, timedelta

# Initial start date
start_date = datetime.strptime('19900101', '%Y%m%d')

# Define the interval (15 days)
interval = timedelta(days=10)

# Initialize an empty list to store the date ranges
date_ranges = []

# Loop to create date intervals
for _ in range(10000):  # Adjust the range to get the desired number of intervals
    end_date = start_date + interval
    date_ranges.append((start_date.strftime('%Y%m%d'), end_date.strftime('%Y%m%d')))
    start_date = end_date + timedelta(days=1)

# Print the date ranges
for start, end in date_ranges:
    success = False
    while not success:
        try:
            print(f"Start Date: {start}, End Date: {end}")

            print("Downloading data from refinitiv...")

            mna_screen = f'SCREEN(U(IN(DEALS)/*UNV:DEALSMNA*/),\
                        BETWEEN(TR.MnAAnnDate,{start},{end})/*dt:Date*/, \
                        NOT_IN(TR.MnAAcquirorNation, "CN"),\
                        NOT_IN(TR.MnAStatus, "S", "R", "DR", "W", "SW"), \
                        CURN=USD)'


            # input ipo screen results as instrument list and pull ipo attributes via fields from Eikon Data API. 
            mnas = rdp.get_data(mna_screen, ['TR.MnASDCDealNumber',
                                           'TR.MnAAnnDate',
                                           'TR.MnARankValueIncNetDebt(Curn=USD,Scale=6)',
                                           'TR.MnADealValue(Curn=USD,Scale=6)', 'TR.MnADealCurrency',
                                           'TR.MnADealType',
                                           'TR.MnATarget', 'TR.MnATargetNation', 'TR.MnATargetSDCCusip',
                                           'TR.MnAAcquiror', 'TR.MnAAcquirorNation', 'TR.MnAAcquirorSDCCusip',
                                           'TR.MnAAcquirorUltParent', 'TR.MnAAcquirorUltParentSDCCusip',
                                           'TR.MnAPctHeldAtAnnDate', 'TR.MnAPctOfSharesAcquired', 'TR.MnAPctOfSharesAcqSeeksToOwn',
                                           'TR.MnAStatus',
                                           'TR.MnACompDealId(Concat="|")']) 
            print(mnas.shape)

            print("Uploading data to BQ...")
            mnas.columns = [col[3:].lower() for col in mnas.columns]

            # Load the DataFrame into BigQuery
            load_job = client.load_table_from_dataframe(mnas, table_ref, job_config=job_config)

            # Wait for the load job to complete
            load_job.result()

            print("Data appended successfully.")

            success = True 
            
        except Exception as e:
            print(f"An error occurred: {e}. Retrying for the range {start} to {end}...")

Start Date: 19900101, End Date: 19900111
An error occurred: 'NoneType' object has no attribute 'shape'. Retrying for the range 19900101 to 19900111...
Start Date: 19900101, End Date: 19900111
An error occurred: 'NoneType' object has no attribute 'shape'. Retrying for the range 19900101 to 19900111...
Start Date: 19900101, End Date: 19900111
An error occurred: 'NoneType' object has no attribute 'shape'. Retrying for the range 19900101 to 19900111...
Start Date: 19900101, End Date: 19900111
An error occurred: 'NoneType' object has no attribute 'shape'. Retrying for the range 19900101 to 19900111...
Start Date: 19900101, End Date: 19900111
An error occurred: 'NoneType' object has no attribute 'shape'. Retrying for the range 19900101 to 19900111...
Start Date: 19900101, End Date: 19900111
An error occurred: 'NoneType' object has no attribute 'shape'. Retrying for the range 19900101 to 19900111...
Start Date: 19900101, End Date: 19900111
An error occurred: 'NoneType' object has no attribute

KeyboardInterrupt: 

In [ ]:
mna_screen = f'SCREEN(U(IN(DEALS)/*UNV:DEALSMNA*/),\
                    BETWEEN(TR.MnAAnnDate,20180101,20180105)/*dt:Date*/, \
                    CURN=USD)'


# input ipo screen results as instrument list and pull ipo attributes via fields from Eikon Data API. 
mnas = rdp.get_data(mna_screen,['TR.MNASDCDealNumber',
                                'TR.MnAAnnDate',
                                'TR.MnARankValueIncNetDebt(Curn=USD,Scale=6)',
                                'TR.MnATarget',
                                'TR.MnATargetPermId',
                                'TR.MnATargetPriTickerSym',
                                'TR.MnATargetNation',
                                'TR.MnATargetMacroInd',
                                'TR.MnATargetMidInd',
                                'TR.MnAAcquiror',
                                'TR.MnAAcquirorPermId',
                                'TR.MnAAcquirorMacroInd',
                                'TR.MnAAcquirorMidInd',
                                'TR.MnAAcquirorNation',
                                'TR.MNASDCDealNumber',
                                'TR.MNADealId','TR.MnASynopsis',
                                'TR.MnAAcquirorPermId',
                                'TR.MnAAcquirorPriHiTech',
                                'TR.MnAAcquirorPriTickerSym',
                                'TR.MnAAcquirorExchTicker(Concat="|")',
                                'TR.MnAAcquirorUltParentPermID',
                                'TR.MnAHiTechGroup(Concat="|")',
                                'TR.MnaAcquirorDate1DayPriorToAnn',
                                'TR.MnaAcquirorStockPrice1DayPriorToAnnInAcqCurr',
                                'TR.MnAAcquirorStockPriceOnAnn(Curn=USD)',
                                'TR.MnAAcquirorStockPrice1DayPostAnn(Curn=USD)',
                                'TR.MnASnP5001DayPreAnn',
                                'TR.MnASnP500AtAnnouncement',
                                'TR.MnASnP5001DayPostAnn',
                                'TR.MnATargetSDCCusip',
                                'TR.MnAAcquirorSDCCusip',
                                'TR.MnAAcquirorUltParentSDCCusip',
                                'TR.MnAValueOfCash(Curn=USD, Scale=6)',
                                'TR.MnAValueOfEarnout(Curn=USD, Scale=6)',
                                'TR.MnADealValue(Curn=USD, Scale=6)',
                                'TR.MnAAcquirorHiTech(Concat="|")',
                                'TR.MnAAcquirorUltParentHiTech(Concat="|")',
                                'TR.MnATargetPriHiTech',
                                'TR.MnAAcquirorMajorInd',
                                'TR.MnAAcquirorMajorIndGrp',
                                'TR.MnAAcquirorPriVEIC',
                                'TR.MnAMasterDealType',
                                'TR.MnADealType',
                                'TR.MnAType(Concat="|")',
                                'TR.MnAAcquirorVeic(Concat="|")',
                                'TR.MnAPurposeDescription',
                                'TR.MnAIsCrossBorder',
                                'TR.MnAAcquirorIsVentureFirm',
                                'TR.MnATargetPriVEIC',
                                'TR.MnATargetVeic(Concat="|")',
                                'TR.MnAAcquirorBusDesc',
                                'TR.MnATargetBusDesc',
                                'TR.MnATargetPermId',
                                'TR.MnATargetExchTicker(Concat="|")',
                                'TR.MnATargetUltParentSDCCusip',
                                'TR.MnATargetUltParentPriTickerSym',
                                'TR.MnATargetUltParentPermID',
                                'TR.MnAAcquirorUltParentPriTickerSym',
                                'TR.MnAAcquirorUltParentExchTicker(Concat="|")',
                                'TR.MnATargetUltParentExchTicker(Concat="|")',
                                'TR.MnATargetMajorInd',
                                'TR.MnATargetMajorIndGrp',
                                'TR.MnATargetHiTech(Concat="|")',
                                'TR.MnAAcquirorUltParentPriHiTech',
                                'TR.MnAAcquirorHiTechGroup(Concat="|")',
                                'TR.MnATargetHiTechGroup(Concat="|")',
                                'TR.MnAWithdrawnDate',
                                'TR.MnADateUnconditional',
                                'TR.MnAOrigAnnDate',
                                'TR.MnAPercentCash',
                                'TR.MnADealCurrency',
                                'TR.MnAEarnoutDesc']) 

print(mnas.shape)
print(mnas.columns)
mnasa

In [ ]:

# Initialize a BigQuery client
client = bigquery.Client()

# Define your project, dataset, and table
project_id = "starfire-410116"
dataset_id = "derived"
table_id = "refinitiv_deals_2"


# Construct a reference to the target table
table_ref = client.dataset(dataset_id).table(table_id)

# Configure the load job to append data to the existing table
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND
)


from datetime import datetime, timedelta

# Initial start date
start_date = datetime.strptime('19900101', '%Y%m%d')

# Define the interval (15 days)
interval = timedelta(days=10)

# Initialize an empty list to store the date ranges
date_ranges = []

# Loop to create date intervals
for _ in range(10000):  # Adjust the range to get the desired number of intervals
    end_date = start_date + interval
    date_ranges.append((start_date.strftime('%Y%m%d'), end_date.strftime('%Y%m%d')))
    start_date = end_date + timedelta(days=1)

# Print the date ranges
for start, end in date_ranges:
    success = False
    while not success:
        try:
            print(f"Start Date: {start}, End Date: {end}")

            print("Downloading data from refinitiv...")

            mna_screen = f'SCREEN(U(IN(DEALS)/*UNV:DEALSMNA*/),\
                        BETWEEN(TR.MnAAnnDate,{start},{end})/*dt:Date*/, \
                        NOT_IN(TR.MnAAcquirorNation, "CN"),\
                        NOT_IN(TR.MnAStatus, "S", "R", "DR", "W", "SW"), \
                        CURN=USD)'


            # input ipo screen results as instrument list and pull ipo attributes via fields from Eikon Data API. 
            mnas = rdp.get_data(mna_screen, ['TR.MnASDCDealNumber',
                                           'TR.MnAAnnDate',
                                           'TR.MnARankValueIncNetDebt(Curn=USD,Scale=6)',
                                           'TR.MnADealValue(Curn=USD,Scale=6)', 'TR.MnADealCurrency',
                                           'TR.MnADealType',
                                           'TR.MnATarget', 'TR.MnATargetNation', 'TR.MnATargetSDCCusip',
                                           'TR.MnAAcquiror', 'TR.MnAAcquirorNation', 'TR.MnAAcquirorSDCCusip',
                                           'TR.MnAAcquirorUltParent', 'TR.MnAAcquirorUltParentSDCCusip',
                                           'TR.MnAPctHeldAtAnnDate', 'TR.MnAPctOfSharesAcquired', 'TR.MnAPctOfSharesAcqSeeksToOwn',
                                           'TR.MnAStatus',
                                           'TR.MnACompDealId(Concat="|")']) 
            print(mnas.shape)

            print("Uploading data to BQ...")
            mnas.columns = [col[3:].lower() for col in mnas.columns]

            # Load the DataFrame into BigQuery
            load_job = client.load_table_from_dataframe(mnas, table_ref, job_config=job_config)

            # Wait for the load job to complete
            load_job.result()

            print("Data appended successfully.")

            success = True 
            
        except Exception as e:
            print(f"An error occurred: {e}. Retrying for the range {start} to {end}...")

In [26]:
# ## This chunk is not used. This is an example code to get daily price data via get_historical... Shares outstanding not available. also, no currency conversion function. I'm using a different function.
# price_hist_example = rdp.get_historical_price_summaries(
#     universe = '.SPX', 
#     interval = rdp.Intervals.DAILY,
#     start = "2024-08-16", 
#     end = "2024-08-16",
#     fields = ['TRDPRC_1', 'ACVOL_UNS',  'OPEN_PRC', 'TRNOVR_UNS',  'NUM_MOVES', 'BLKVOLUM']
# )

# print(price_hist_example)



# ## Example function I use to retrieve stock data for a given date. This allows me to retrieve to shares outstanding, currency == USD
# ## if want native curreny pass: CF_LAST, CF_CURR
# ## totalReturn1D is for the previous day? https://community.developers.refinitiv.com/questions/103917/total-return-for-the-same-day.html

# rdp.get_data(
#     'AAPL.O', 
#     fields = ['TR.CLOSEPRICE.Date','TR.CLOSEPRICE', 'TR.PriceOpen', 
#               'TR.VOLUME', 'TR.SharesOutstanding', 'TR.CompanyMarketCap', 
#               'TR.TotalReturn1D'],
#     parameters = {'SDate':'2024-08-10','EDate':'2024-08-16','Curn':'USD'}
# )


## Example to get index pr

# index_ticker = '.DJI'
#  # ['.DJI', '.FTSE', '.SPX', '.STOXX50E', '.HSI', '.BVSP']

rdp.convert_symbols(
    ['.AXJO'], 
    from_symbol_type='RIC')

dab = rdp.get_data(
    '.AXJO',
    fields = ['TR.CLOSEPRICE.Date','TR.CLOSEPRICE', 'TR.PriceOpen', 
              'TR.VOLUME'],
    parameters = {'SDate':'2010-08-10','EDate':'2024-08-22','Curn':'USD'}
)
dab


 


,instrument,Date,TR.CLOSEPRICE,TR.PriceOpen,TR.Volume
0,.AXJO,2010-08-10T00:00:00,4147.932291,4195.525674,1074180923
1,.AXJO,2010-08-11T00:00:00,3992.13311,4059.960397,1000399859
2,.AXJO,2010-08-12T00:00:00,3942.752195,3974.91489,1387994857
3,.AXJO,2010-08-13T00:00:00,3981.536868,3929.84367,1150097008
4,.AXJO,2010-08-16T00:00:00,3985.757774,3994.917339,947561735
...,...,...,...,...,...
3658,.AXJO,2024-08-16T00:00:00,5313.535313,5243.142352,673911542
3659,.AXJO,2024-08-19T00:00:00,5369.197957,5362.940935,611421953
3660,.AXJO,2024-08-20T00:00:00,5391.247489,5379.585563,<NA>
3661,.AXJO,2024-08-21T00:00:00,5400.676897,5392.04714,<NA>


In [20]:
dab.to_csv('sp500_history.csv')

In [ ]:
# get list of exchange countries


RIC = 'BDMN.JK'
RIC = 'MSFT.O'
rics = merged_df['RIC'].str.strip()  # Remove leading/trailing spaces
print(rics)
start_date = '2019-01-11'
end_date = '2019-01-25'


# ric_returns = rdp.get_data(RIC,
#             fields = ['TR.CLOSEPRICE.Date','TR.CLOSEPRICE', 'TR.PriceOpen', 
#                                 'TR.VOLUME', 'TR.SharesOutstanding', 'TR.CompanyMarketCap', 
#                                 'TR.TotalReturn1D', 'TR.MnARegionHQ(Concat="|")'],
#             parameters = {'SDate':start_date,
#                             'EDate':end_date,
#                             'Curn':'USD'}
#             )
# print(ric_returns)

market = rdp.get_data(RIC,
            fields = ['TR.HQCountryCode', 'TR.HeadquartersRegion',  'TR.HQMinorRegion', 'TR.ExchangeRegion', 'TR.ExchangeCountry','TR.ExchangeCountryCode' ]
            )

exch_countries = pd.DataFrame()
counter = 0
for ric in historical_data['RIC'].unique():
    success = False
    
    while not success:
        try:
            counter += 1
            print(f"Processing RIC: {ric}, {counter}")
            market = rdp.get_data(ric, fields=['TR.ExchangeCountryCode'])
            exch_countries = pd.concat([exch_countries, market], ignore_index=True)
            success = True  # If the operation is successful, set success to True to exit the loop
        except Exception as e:
            print(f"Error encountered for RIC {ric}: {e}")
            print("Retrying...")

# At this point, exch_countries should have all the data after successful attempts